In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.patches as mpatches
import numpy as np
import math

import mahotas as mh
import cv2
from PIL import Image

import copy, os

from skimage import io, exposure
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, opening, square, disk
from skimage.draw import disk as pdisk
from skimage.color import label2rgb, rgb2gray

from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max

from skimage.filters import threshold_local, threshold_otsu, threshold_multiotsu, rank
from skimage.filters import sobel

from scipy.stats import skew
from scipy.signal import argrelextrema


In [2]:
PRECISION = 30

In [3]:
class Approximation(): 
    def __init__(self, reg_props, approx_patch, intensity,  idno):
        self.reg_props = reg_props
        self.approx_patch = approx_patch
        self.intensity = intensity
        self.idno = idno

In [4]:
def background_filter(flattened_img):
    hist, bins = np.histogram(flattened_img, pow(PRECISION, 2)/20)
    mode_idx = np.argmax(hist)
    mode = bins[mode_idx]
    return mode

In [5]:
def reflect(image_ts):
    height = image_ts.shape[0]
    image_ts = np.transpose(image_ts)
    to_ret = np.copy(image_ts)
    for col_no, column in enumerate(image_ts):
        nonzero = False
        idx = 0
        while not nonzero and idx < height:
            nonzero = column[idx] > 0
            idx += 1
        start_idx = idx - 1
        while nonzero and idx < height:
            nonzero = column[idx] > 0
            idx += 1
        stop_idx = idx - 1
        reflection = np.flip(column[start_idx:stop_idx])
        reflect_offset_up = start_idx - len(reflection)
        reflect_offset_down = stop_idx + len(reflection)
        to_ret[col_no][max(0, reflect_offset_up):start_idx] = reflection[max(0, -reflect_offset_up):]
        to_ret[col_no][stop_idx:min(reflect_offset_down, height)] = reflection[:min(len(reflection)+1, height - stop_idx)]
    return np.transpose(to_ret)

In [6]:
def calculate_ieg_segm_params(image_ts):
    
    reflected = reflect(image_ts)
    background = ndi.generic_filter(reflected, background_filter, size=PRECISION)
    skewness = skew(image_ts[image_ts>0])
    
    
    return threshold_otsu(image_ts[image_ts>0]), background, skewness

In [7]:
def ieg_segmentation(image_ts, thresh_down, thresh_up, thresh_range, mask): 

        
    elev_map = sobel(image_ts)
    distance = ndi.distance_transform_edt(mask)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3,3)),
                                labels=mask)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-elev_map, markers, mask=mask)
    label_image = label(labels, connectivity = 1)
    
    
    masked = np.ma.masked_array(data = image_ts, mask = mask, fill_value = 0).filled()

    intensities_normalized = np.array([])
    dots = np.array([])
    
    for region in regionprops(label_image, intensity_image=image_ts):
        areas = np.append(areas, region.area)
        reg_app_w_bgr = pdisk(region.centroid, 7, shape = image_ts.shape)
        reg_approx = pdisk(region.centroid, 5, shape = image_ts.shape)
        reg_intensity = 2*np.sum(image_ts[reg_approx])-np.sum(image_ts[reg_app_w_bgr])
        circ = mpatches.Circle((region.centroid[1], region.centroid[0]), 5, 
                               fill=False, edgecolor=color, linewidth=2)
        new_dot = Approximation(region, copy.copy(circ), reg_intensity, len(dots))
        intensities_normalized = np.append(intensities_normalized, reg_intensity)



    return dots, intensities_normalized

In [8]:
scan_path_root = "/home/ula/catfish/test_scans/"

suffix = "/2.tif"
overlay_suffix = "/overlay2.tif"

In [ ]:
#MAIN loop to walk through img folder, looking for FOLDERS

for filename in os.listdir(scan_path_root):
    if os.path.isdir(scan_path_root + filename):
        print(scan_path_root + filename)
        img_path = scan_path_root + filename + suffix
        ieg_img = io.imread(img_path)
        thre, background, skewness = calculate_ieg_segm_params(ieg_img)
        break;

/home/ula/catfish/test_scans/CA1_left_glass_1_rat_2_hipp_left_skan_2019-10-08_105903


In [ ]:
pow(50, 2)/20

In [ ]:
np.array([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]])[:3,:3]